In [4]:
import duckdb
import imageio
import numpy as np
import shutil
from PIL import Image
min_date = '2022-04-01 12'

color_name = {
    16729344: 'red', 
    0: 'black', 
    6970623: 'periwinkle', 
    7143450: 'burgandy', 
    16775352: 'pale yellow', 
    2379940: 'dark blue', 
    8461983: 'dark purple', 
    16754688: 'orange', 
    40618: 'teal', 
    52416: 'light teal', 
    8318294: 'light green', 
    12451897: 'dark red', 
    16757872: 'beige', 
    9745407: 'lavendar', 
    16777215: 'white', 
    11815616: 'purple', 
    41832: 'dark green', 
    10250534: 'brown', 
    14986239: 'pale purple', 
    16766517: 'yellow', 
    3576042: 'blue', 
    16751018: 'light pink', 
    5368308: 'light blue', 
    9014672: 'gray', 
    4799169: 'indigo', 
    30063: 'dark teal', 
    13948889: 'light gray', 
    5329490: 'dark gray', 
    52344: 'green', 
    16726145: 'pink', 
    7161903: 'dark brown', 
    14553215: 'magenta'
}

db_path = r"../data/lossy_2022_place_history.parquet"
db = duckdb.connect(":memory:")
db.sql(f"CREATE TABLE data AS SELECT * FROM read_parquet('{db_path}')")

In [2]:
# Utility Functions
# Convert a day and hour difference into purely an hour one
def min_hour_delta(day : int, hour : int):
    min_tokens = min_date.split(' ')
    # Get Hour Difference
    hour_diff = hour - int(min_tokens[1])
    # Get Day Difference
    day_hour_diff = 24 * (day - int((min_tokens[0].split('-'))[2]))
    hour_delta = hour_diff + day_hour_diff
    return hour_delta

# Returns rgb values in a list from an integer representation of hex code
def int_to_rgb(color_value : int) -> list[int]:
    hex_code = f"{color_value:06x}" # Convert To Hex
    return [ int(hex_code[0:2], 16), int(hex_code[2:4], 16), int(hex_code[4:6], 16)]

In [32]:
# Analysis Functions
# Draw most used colors around a coordinate up to a specific hour or only during a specific hour
def img_most_used_near(x : int, y : int, size : int, hour : int, mode : int):
    x_lower_bound = (x-size) if x > size else 0
    x_upper_bound = (x+size) if (x-size) < 2000 else 2000
    y_lower_bound = (y-size) if y > size else 0
    y_upper_bound = (y+size) if (y-size) < 2000 else 2000

    # Bias towards brighter colors
    condition_str = ""
    # Mode 0 Shows Current Top Placements -> Bias Towards High Activity 
    if (mode == 0):
        condition_str += f"WHERE timedelta = {hour} ORDER BY color"
    # Mode 1 Shows Most Recent Top Placements -> Bias Towards Low Activity
    elif (mode == 1):
        condition_str += f"WHERE timedelta <= {hour} ORDER BY timedelta, color"

    sql_to_execute = f"""
        WITH mark AS (
            SELECT x, y, color, timedelta, COUNT(1) AS amt FROM data
            WHERE ((x < {x_upper_bound} AND x >= {x_lower_bound}) 
            AND (y < {y_upper_bound} AND y >= {y_lower_bound})) 
            AND (timedelta >= {min} AND timedelta <= {max})
            GROUP BY x, y, color, timedelta
        ), top_color AS (
            SELECT x, y, timedelta, MAX(amt) AS top FROM mark GROUP BY x, y, timedelta
        ), agg AS (
            SELECT mark.x, mark.y, color, mark.timedelta FROM mark 
            JOIN top_color ON mark.timedelta = top_color.timedelta 
            AND mark.amt = top_color.top AND mark.x = top_color.x AND mark.y = top_color.y
        )

        SELECT color, x, y FROM agg {condition_str}
    """

    coords = db.sql(sql_to_execute).fetchall()

    width = x_upper_bound - x_lower_bound
    height = y_upper_bound - y_lower_bound
    img_set = np.full([height, width, 3], 50, dtype=np.uint8) # fill with dark grey to show black
    for coord in coords:
        img_set[coord[2]-y_lower_bound][coord[1]-x_lower_bound] = int_to_rgb(coord[0])

    img = Image.fromarray(img_set, "RGB")
    file_name = f"{x}_{y}_{size}_{hour}_mode_{mode}.png"
    img.save(file_name)
    shutil.move(file_name, f"visualizations/pngs/{file_name}")
    img_set = None
    #print(file_name)

# Animate image near specific coordinates within a specific time frame
def animate_at_(x : int, y : int, size : int, mode : int, min : int, max : int):
    for i in range(min, max):
        img_most_used_near(x, y, size, i, mode)
    img_set = [imageio.imread(f"visualizations/pngs/{x}_{y}_{size}_{i}_mode_{mode}.png") for i in range(min, max)]
    file_name = f"{x}_{y}_{size}_animate_{mode}_{min}_{max}.gif"
    imageio.mimsave(file_name, img_set, format="GIF", fps=3)
    shutil.move(file_name, f"visualizations/{file_name}")

# Return top 3 most used colors at a coordinate
def top_3_color_at(x : int, y : int, min : int, max : int):
    coords = db.sql(f"""
        SELECT color, COUNT(1) AS amount FROM data 
        WHERE 
        timedelta >= {min} AND timedelta <= {max}
        AND (x = {x} AND y = {y})
        GROUP BY color
        ORDER BY amount DESC
        LIMIT 3
    """).fetchall()

    for coord in coords:
        print(f"Coord: ({x}, {y}) {coord[1]} {color_name[int(coord[0])]}")

# Return top 10 most active hours
def top_10_activity_at(x : int, y: int):
    coords = db.sql(f"""
        WITH activity AS (
            SELECT timedelta, COUNT(1) AS act FROM data 
            WHERE (x = {x} AND y = {y})
            GROUP BY timedelta
        )

        SELECT timedelta, act FROM activity ORDER BY act DESC LIMIT 10
    """).fetchall()

    for i, coord in enumerate(coords):
        print(f"{i+1}. {coord}")

# Count Activity At Coordinate
def activity_at(x : int, y : int, min : int, max : int):
    activity = db.sql(f"""
        SELECT COUNT(1) AS act FROM data WHERE x = {x} AND y = {y} AND timedelta <= {max} AND timedelta >= {min}
    """).fetchone()
    print(f"({x}, {y}). {activity[0]}")


In [6]:
# Find 3 Most Placed Pixels
min = min_hour_delta(1, 12)
max = min_hour_delta(1, 96)

coord = db.sql(f"SELECT x, y, COUNT(1) AS amt FROM data WHERE timedelta >= {min} AND timedelta <= {max} GROUP BY x, y ORDER BY COUNT(1) DESC LIMIT 3").fetchall()

for i in range(3):
    print(f"{i+1}. Coord: {coord[i]}")


1. Coord: (0, 0, 98807)
2. Coord: (359, 564, 69198)
3. Coord: (349, 564, 55230)


In [106]:
top_3_color_at(0, 0, min, max)
top_3_color_at(359, 564, min, max)
top_3_color_at(349, 564, min, max)

Coord: (0, 0) 59282 white
Coord: (0, 0) 8715 black
Coord: (0, 0) 4209 red
Coord: (359, 564) 34726 black
Coord: (359, 564) 26940 light blue
Coord: (359, 564) 1656 red
Coord: (349, 564) 27804 black
Coord: (349, 564) 19404 light blue
Coord: (349, 564) 2120 red


In [176]:
# Animated Only Once At (349, 564) Due to Close Proximity Of (359, 564)
# One piece skeleton
animate_at_(349, 564, 35, 0, min, max)
animate_at_(349, 564, 35, 1, min, max)


/var/folders/1x/dycpz7w970v75zp6v07k7_zr0000gn/T/ipykernel_75609/2292213344.py:55: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_set = [imageio.imread(f"visualizations/pngs/{x}_{y}_{size}_{i}_mode_{mode}.png") for i in range(min, max)]


In [175]:
# Corner of Runescape Sign
animate_at_(0, 0, 35, 0, min, max)
animate_at_(0, 0, 35, 1, min, max)

/var/folders/1x/dycpz7w970v75zp6v07k7_zr0000gn/T/ipykernel_75609/2292213344.py:55: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_set = [imageio.imread(f"visualizations/pngs/{x}_{y}_{size}_{i}_mode_{mode}.png") for i in range(min, max)]


In [ ]:
# Animatation of One Piece Skeleton Eyes
animate_at_(349, 564, 5, 0, min, max)
animate_at_(359, 564, 5, 0, min, max)

/var/folders/1x/dycpz7w970v75zp6v07k7_zr0000gn/T/ipykernel_75609/2292213344.py:55: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_set = [imageio.imread(f"visualizations/pngs/{x}_{y}_{size}_{i}_mode_{mode}.png") for i in range(min, max)]


In [208]:
# Larger Portion of Runescape Please Wait Sign
animate_at_(0, 0, 100, 1, min, max)

/var/folders/1x/dycpz7w970v75zp6v07k7_zr0000gn/T/ipykernel_75609/4246140971.py:55: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_set = [imageio.imread(f"visualizations/pngs/{x}_{y}_{size}_{i}_mode_{mode}_1.png") for i in range(min, max)]


In [197]:
top_10_activity_at(0, 0)
top_10_activity_at(349, 564)
top_10_activity_at(359, 564)

1. (84, 15618)
2. (83, 12212)
3. (79, 1597)
4. (80, 1586)
5. (78, 1570)
6. (82, 1563)
7. (77, 1529)
8. (56, 1405)
9. (81, 1396)
10. (75, 1372)
1. (54, 1049)
2. (53, 1043)
3. (31, 1027)
4. (32, 1015)
5. (34, 986)
6. (52, 959)
7. (51, 937)
8. (33, 933)
9. (39, 925)
10. (40, 923)
1. (53, 1321)
2. (54, 1250)
3. (32, 1204)
4. (34, 1180)
5. (33, 1160)
6. (31, 1147)
7. (38, 1142)
8. (52, 1134)
9. (39, 1129)
10. (35, 1098)


In [8]:
# Top Left Corner
activity_at(0, 0, min, max)

# Top Right Corners
activity_at(0, 999, min, max)
activity_at(0, 1999, min, max)

# Bottom Left Corners 
activity_at(999, 0, min, max)
activity_at(1999, 0, min, max)

# Bottom Right Corners
activity_at(999, 999, min, max)
activity_at(1999, 1999, min, max)

(0, 0). 98807
(0, 999). 22358
(0, 1999). 22763
(999, 0). 8433
(1999, 0). 30882
(999, 999). 23271
(1999, 1999). 31437


In [12]:
top_10_activity_at(0, 1999)
top_10_activity_at(1999, 1999)
top_10_activity_at(1999, 0)

1. (83, 2964)
2. (84, 2378)
3. (55, 1141)
4. (81, 1036)
5. (82, 969)
6. (80, 891)
7. (78, 889)
8. (77, 853)
9. (79, 821)
10. (76, 752)
1. (84, 3455)
2. (83, 2426)
3. (55, 1329)
4. (78, 1319)
5. (80, 1310)
6. (81, 1307)
7. (79, 1301)
8. (77, 1215)
9. (76, 1135)
10. (75, 1123)
1. (84, 2217)
2. (83, 1547)
3. (80, 948)
4. (79, 919)
5. (78, 905)
6. (77, 843)
7. (74, 836)
8. (81, 833)
9. (58, 721)
10. (82, 699)
